In [ ]:
#Notebook for development of the game
# Imports main modules - Player, MapGenerator, etc.
# Implements pygame controls and GUI

In [ ]:
import sys

#where non-dev code is located
sys.path.append('../app')

In [3]:
#install pygame
!{sys.executable} -m pip install pygame

You should consider upgrading via the 'c:\ml\env\scripts\python.exe -m pip install --upgrade pip' command.


In [4]:
#config stores constants and default settings
from config import Config

#helpful pygame wrapper
import pygame_utils as pyg

pygame 2.0.1 (SDL 2.0.14, Python 3.9.1)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [ ]:
# Game GUI consists of:
# - menu through which the user chooses a few settings the starts the game
# - game screen consisting of
#   - the map and players
#   - some type of status bar on the bottom or side allowing restart and showing info
# - winner outcome screen which displays at game end for a while before going back to the menu

In [ ]:
#untested code skeleton
#class for menu GUI elements

class Menu():
    def __init__(self, config, map_generator):
        self.config = config
        
        #holds a generated map, can replace generated map with .generate_map()
        self.map_generator = map_generator
        
        #the dict of values populated from the menu
        self.menu_values = {}
        
        self.menu_background = pyg.makeSprite(config.menu_background_image_path)
        pyg.moveSprite(self.menu_background, 200,150) #TODO - location
        pyg.showSprite(self.menu_background)
            
        self.text_label = pyg.makeLabel("Capture the Flag", 22, 200, 50, "white", font="impact")#TODO - location
        pyg.showLabel(self.text_label)
        
        #elements of the menu
        self.game_mode_select = pyg.makeSprite(config.menu_game_mode_select_image_path)
        pyg.moveSprite(self.game_mode_select, 200,150)#TODO - location
        pyg.showSprite(self.game_mode_select)
        
        self.team_size_select = pyg.makeSprite(config.menu_team_size_select_image_path)
        pyg.moveSprite(self.team_size_select, 200,150)#TODO - location
        pyg.showSprite(self.team_size_select)
            
        self.regenerate_map_button = pyg.makeSprite(config.menu_regenerate_map_button_image_path)
        pyg.moveSprite(self.regenerate_map_button, 200,150)#TODO - location
        pyg.showSprite(self.regenerate_map_button)
            
        self.start_game_button = pyg.makeSprite(config.menu_start_game_button_image_path)
        pyg.moveSprite(self.start_game_button, 200,150)#TODO - location
        pyg.showSprite(self.start_game_button)
            
        #display small version of generated map so user can choose to regenerate it
        #TODO use the menu_image in map_generator to paste on the sprite
        self.map_display = pyg.makeSprite(config.menu_map_display_image_path)
        pyg.moveSprite(self.map_display, 200,150)#TODO - location
        pyg.showSprite(self.map_display)
        
    
    def get_values(self):
        '''When Start Game button is pressed, returns dict of values populated from the menu.'''
        game_mode = self.config.game_mode_play
        team_size = self.config.team1_size
        player_team = self.config.team1_name
        
        while True:
            #display menu for choosing game mode, number of players, team, map
            if pyg.spriteClicked(self.game_mode_select):
                self.menu_values['game_mode'] = self.get_game_mode()
                
            elif pyg.spriteClicked(self.team_size_select):
                self.menu_values['team_size'] = self.get_team_size()
                
            elif pyg.spriteClicked(self.player_team_select):
                self.menu_values['player_team'] = self.get_player_team()
                
            elif pyg.spriteClicked(self.regenerate_map_button):
                self.map_generator.generate_map()
                self.update_map(self.map_generator.menu_image)
                
            elif pyg.spriteClicked(self.start_game_button):
                #destroy the menu and return the values
                pyg.hideLabel(self.text_label)
                pyg.killSprite(self.game_mode_select)
                pyg.killSprite(self.team_size_select)
                pyg.killSprite(self.regenerate_map_button)
                pyg.killSprite(self.start_game_button)
                pyg.killSprite(self.map_display)
                pyg.killSprite(self.menu_background)
                return self.menu_values
            
            tick(config.menu_frame_rate)
            
                
    def get_game_mode(self):
        #TODO get from selection
        return self.config.game_mode_play
    
    def get_team_size(self):
        #TODO get from selection
        return self.config.team1_size
    
    def get_player_team(self):
        #TODO get from selection
        return self.config.team1_name
    
    def update_map(self, im):
        #TODO update map image sprite
        pass

In [ ]:
#untested code skeleton
# class for game status window next to map

class GameStatusWindow():
    def __init__(self):
        pass
    def display(self):
        pass

In [ ]:
#untested code skeleton
#class for map GUI elements

class GameMapWindow():
    def __init__(self, game_status_window):
        pass
    def display(self):
        pass

In [ ]:
#untested code skeleton
#top class for creating all GUI elements and running the game

class Game():
    def __init__(self, config):
        self.config = config
        
        #instantiate pygame objects using game vars from config
        pyg.screenSize(config.screen_width, config.screen_height)
        
        pyg.setIcon(config.game_icon_image_path)
        pyg.setWindowTitle(config.game_name)
        pyg.setBackgroundImage(config.game_background_image_path)
        
        # menu
        
        #map may be regenerated in menu, so access map_generator through menu object
        map_generator = MapGenerator()
        map_generator.generate_map()
        
        self.menu = Menu(config, map_generator)
        
        
    def play(self):
        #main game loop
        while True:
            #wait for values from the menu - currently includes game_mode, team_size, player_team
            menu_values = self.menu.get_values()
            game_mode = menu_values['game_mode']
            team_size = menu_values['team_size']
            player_team = menu_values['player_team']
            
            #the generated map is accessible through the map generator in menu
            #update background with generated map
            #self.menu.map_generator
            
            #create player and agents
            
            #update sprites according to their actions
            
            #check for win

            pyg.tick(config.time_delay)
            
        pyg.endWait()

In [ ]:
config = Config()
game = Game(config)
game.play()